In [1]:
import pandas as pd

In [2]:
from edhec_risk_kit_111 import *

In [3]:
industry_data = pd.read_csv('ind30_m_vw_rets.csv')
industry_data['Unnamed: 0'] = pd.to_datetime(industry_data['Unnamed: 0'], format='%Y%m')
industry_data.rename(columns={'Unnamed: 0': 'date'}, inplace=True)
industry_data.set_index('date', inplace=True)

industry_data

,Food,Beer,Smoke,Games,Books,Hshld,Clths,Hlth,Chems,Txtls,...,Telcm,Servs,BusEq,Paper,Trans,Whlsl,Rtail,Meals,Fin,Other
date,,,,,,,,,,,,,,,,,,,,,
1926-07-01,0.56,-5.19,1.29,2.93,10.97,-0.48,8.08,1.77,8.14,0.39,...,0.83,9.22,2.06,7.70,1.93,-23.79,0.07,1.87,0.37,5.20
1926-08-01,2.59,27.03,6.50,0.55,10.01,-3.58,-2.51,4.25,5.50,8.14,...,2.17,2.02,4.39,-2.38,4.88,5.39,-0.75,-0.13,4.46,6.76
1926-09-01,1.16,4.02,1.26,6.58,-0.99,0.73,-0.51,0.69,5.33,2.31,...,2.41,2.25,0.19,-5.54,0.05,-7.87,0.25,-0.56,-1.23,-3.86
1926-10-01,-3.06,-3.31,1.06,-4.76,9.47,-4.68,0.12,-0.57,-4.76,1.00,...,-0.11,-2.00,-1.09,-5.08,-2.64,-15.38,-2.20,-4.11,-5.16,-8.49
1926-11-01,6.35,7.29,4.55,1.66,-5.80,-0.54,1.87,5.42,5.20,3.11,...,1.63,3.77,3.64,3.84,1.60,4.67,6.52,4.33,2.24,4.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2018-08-01,-0.38,-1.86,-5.43,2.89,-4.47,2.54,5.26,4.46,0.01,3.80,...,2.95,5.24,9.93,-0.34,2.91,3.66,9.11,3.64,2.45,2.99
2018-09-01,-0.32,0.19,5.37,-0.09,-2.21,1.07,1.30,1.99,-2.87,-6.38,...,1.74,-0.37,-0.33,-0.30,1.05,-1.48,0.61,2.51,-1.93,1.16
2018-10-01,1.02,-1.57,7.90,-15.96,-6.66,-0.51,-10.14,-8.84,-12.50,-25.79,...,-0.50,-9.20,-8.06,-9.82,-9.75,-7.88,-10.21,-1.71,-5.45,-5.99


In [4]:
# Load the data using the user-defined function
ind_returns = get_ind_returns()

# Filter data for the required industries and time periods
selected_industries = ["Books", "Steel", "Oil", "Mines"]
data_2013_2017 = ind_returns.loc["2013":"2017", selected_industries]

# Compute expected returns and covariance matrix for 2013–2017
expected_returns_2013_2017 = data_2013_2017.mean()
cov_matrix_2013_2017 = data_2013_2017.cov()

# Assume the risk-free rate over the 5-year period is 10% annualized
risk_free_rate = 0.10

# Compute the MSR portfolio weights using the user-defined function
msr_weights_2013_2017 = msr(risk_free_rate, expected_returns_2013_2017, cov_matrix_2013_2017)

# Filter data for 2018
data_2018 = ind_returns.loc["2018", selected_industries]

# Compute the covariance matrix for 2018
cov_matrix_2018 = data_2018.cov()

# Compute the annualized volatility of the MSR portfolio over 2018
volatility_2018 = portfolio_vol(msr_weights_2013_2017, cov_matrix_2018)
annualized_volatility_2018 = annualize_vol(volatility_2018, periods_per_year=12) * 100

annualized_volatility_2018

0.0

In [18]:
import numpy as np

# Target volatility
target_volatility = 0.05

# Calculate the inverse of the covariance matrix
inv_cov_matrix = np.linalg.inv(covariance_matrix)

# Compute weights for the minimum variance portfolio
ones = np.ones(len(expected_returns))
min_var_weights = inv_cov_matrix @ ones / (ones.T @ inv_cov_matrix @ ones)

# Rescale weights to meet the target volatility
portfolio_volatility = np.sqrt(min_var_weights.T @ covariance_matrix @ min_var_weights)
scaling_factor = target_volatility / portfolio_volatility
optimal_weights = min_var_weights * scaling_factor

# Find the largest weight in percentage terms
largest_weight = np.max(optimal_weights) * 100

round(largest_weight, 2)

0.56

In [19]:
# Risk-free rate
risk_free_rate = 0.10 / 12  # Convert annual rate to monthly

# Calculate excess returns (mean returns above risk-free rate)
excess_returns = expected_returns - risk_free_rate

# Compute weights for the Maximum Sharpe Ratio (MSR) portfolio
msr_weights = inv_cov_matrix @ excess_returns / (ones.T @ inv_cov_matrix @ excess_returns)

# Identify the component with the largest weight
largest_weight_component = selected_industries[np.argmax(msr_weights)]
largest_weight_component

'Oil'

In [7]:
# Adjust risk-free rate
risk_free_rate = 0.03 / 12  # Convert annual rate to monthly

# Calculate excess returns (mean returns above risk-free rate)
excess_returns = expected_returns - risk_free_rate

# Compute weights for the tangency (MSR) portfolio
tangency_weights = inv_cov_matrix @ excess_returns / (ones.T @ inv_cov_matrix @ excess_returns)

# Compute the volatility of the tangency portfolio
tangency_volatility = np.sqrt(tangency_weights.T @ covariance_matrix @ tangency_weights)  # Convert to percent

round(tangency_volatility, 2)

5.61

In [21]:
# Calculate the expected return of the portfolio with target volatility
portfolio_expected_return = optimal_weights.T @ expected_returns

# Convert to percent
portfolio_expected_return_percent = portfolio_expected_return * 100

round(portfolio_expected_return_percent, 2)

0.85

In [9]:
# Compute weights for the Global Minimum Variance (GMV) portfolio
gmv_weights = inv_cov_matrix @ ones / (ones.T @ inv_cov_matrix @ ones)

# Identify the component with the largest weight in GMV portfolio
largest_weight_component_gmv = selected_industries[np.argmax(gmv_weights)]
largest_weight_component_gmv

'Oil'

In [10]:
# Compute equal weights for the portfolio
num_assets = len(selected_industries)
equal_weights = np.full(num_assets, 1 / num_assets)

# Calculate the volatility of the Equal-Weighted (EW) portfolio
ew_portfolio_volatility = np.sqrt(equal_weights.T @ covariance_matrix @ equal_weights)  # Convert to percent

round(ew_portfolio_volatility, 1)


6.2

In [11]:
# Find the largest weight in the GMV portfolio in percentage terms
largest_gmv_weight = np.max(gmv_weights) * 100

round(largest_gmv_weight, 1)


62.1

In [20]:
# Filter data for 2018 to compute realized volatility
data_2018 = industry_data['2018-01':'2018-12']

# Select relevant industries
data_2018 = data_2018[selected_industries]

# Handle missing values and convert to numeric
data_2018 = data_2018.apply(pd.to_numeric, errors='coerce').dropna()

# Compute the covariance matrix for 2018
cov_matrix_2018 = data_2018.cov()

# Compute the portfolio volatility for MSR portfolio weights in 2018
msr_portfolio_volatility_2018 = np.sqrt(msr_weights.T @ cov_matrix_2018 @ msr_weights)

# Annualize the volatility
annualized_volatility_2018 = msr_portfolio_volatility_2018 * np.sqrt(12) * 100  # Convert to percent

round(annualized_volatility_2018, 2)

2136.32

In [22]:
# Filter data for the 2013-2017 period to estimate the MSR portfolio
data_2013_2017 = industry_data['2013-01-01':'2017-12-01']
data_2013_2017 = data_2013_2017[selected_industries]

# Compute expected returns and covariance matrix for 2013-2017
expected_returns_2013_2017 = data_2013_2017.mean()
cov_matrix_2013_2017 = data_2013_2017.cov()

# Compute excess returns for MSR portfolio (risk-free rate = 10% annualized)
risk_free_rate = 0.10 / 12  # Monthly risk-free rate
excess_returns_2013_2017 = expected_returns_2013_2017 - risk_free_rate

# Compute MSR portfolio weights
inv_cov_matrix_2013_2017 = np.linalg.inv(cov_matrix_2013_2017)
msr_weights_2013_2017 = (
    inv_cov_matrix_2013_2017 @ excess_returns_2013_2017
) / (ones.T @ inv_cov_matrix_2013_2017 @ excess_returns_2013_2017)

# Filter data for 2018 to compute realized volatility
data_2018 = industry_data['2018-01-01':'2018-12-01']
data_2018 = data_2018[selected_industries]

# Compute covariance matrix for 2018
cov_matrix_2018 = data_2018.cov()

# Compute annualized volatility of the MSR portfolio in 2018
msr_portfolio_volatility_2018 = np.sqrt(msr_weights_2013_2017.T @ cov_matrix_2018 @ msr_weights_2013_2017)
annualized_volatility_2018 = msr_portfolio_volatility_2018 * np.sqrt(12) * 100  # Convert to percent

round(annualized_volatility_2018, 2)

2385.85